Our goal is to create a 3d puzzle grid :  
--> of size **size_Puzzle**
--> with a number of pieces **number_Of_Pieces**

A *piece* is a part of the matrix that is named by a number (eg piece 5 will be defined by all the location in a 3d matrix where the number 5 can be found). As in real 3d puzzle a piece is continuous, this means that the location that defines the piece are adjacent to each others along 1 axis (see the example picture for better visual understanding).
The *size of the piece* is the number of location taken by this piece in the matrix.


Step 0 : To do that the algortihm select number_Of_Pieces locations in a 3d matrix filled with 0 and write the number of the piece in the matrix. (For 7 pieces there will be 7 locations in the matrix that won't be 0 and be 1,2,3,4,5,6 or 7; these numbers are the "name of the pieces").

Step 1:
While there is still 0 in the 3d matrix :
    Do : Then at each step the algorithm tries to extend the piece to another adjacent location that is not selected yet (with a probability of success).
                If there is at least 1 free location and if the extend is a success then the number of the piece will replace the 0 that was written in the free location
Step 2: Once the expansion of the pieces is over the function check_stat calculate statistics properties of the generated 3d puzzle. If the statitics meets requirement then the puzzle is validated, else the algorithm goes back to Step 0.

In [3]:
import numpy as np
import random
import time
import json

def find_neighboor(matrix:np.array,index:np.array):
    """Find the free adjacent location around a location(index) in the matrix(matrix)

    Index is an array with 3 coordinates (as it is a 3d matrix)
    """
    neighboor=np.zeros(6)
    for i in range(0,3): #3d matrix so 3 axis
        index_forw=tuple(index+np.where(np.arange(3)==i,1,0))
        index_backw=tuple(index-np.where(np.arange(3)==i,1,0))
        if index_forw[i]<matrix.shape[0]:
            if matrix[index_forw]==0:
                neighboor[2*i]=1
        if index_backw[i]>=0:
            if matrix[index_backw]==0:
                neighboor[2*i+1]=1
    return neighboor #All available positions



def check_stat(dic):
    """Assesses the mean size of pieces and the standard deviation    
    """
    sizes_pieces=[]
    for key in dic.keys():
        piece=dic[key]
        size_piece=len(piece)
        sizes_pieces.append(size_piece)
    return np.mean(sizes_pieces),np.std(sizes_pieces)


def generate_puzzle(size_Puzzle : int,number_Of_Pieces:int):
    """Generate a 3d puzzle grid with a given size of 3d matrix and a given number of pieces
    
        For Complexity reason the minimum size of a piece is enforced to 2 (because 1 size cubes would be to easy to fit in the puzzle)
        The function returns the puzzle grid and a dictionnary that summarizes all the location belonging to each piece
    """
    dic_pieces={}
    sucess_2_cube=False
    while sucess_2_cube!=True:
        mat_preselect=np.zeros((size_Puzzle**3))
        index_start_pieces=random.sample(range(size_Puzzle**3),number_Of_Pieces)
        for k,index in zip(range(1,number_Of_Pieces+1),index_start_pieces):
            mat_preselect[index]=k
        sucess_2_cube=True
        puzzle_grid=mat_preselect.reshape((size_Puzzle,size_Puzzle,size_Puzzle))
        for k,index in zip(range(1,number_Of_Pieces+1),index_start_pieces):
            position=(index//(size_Puzzle**2),(index%(size_Puzzle**2))//size_Puzzle,(index%size_Puzzle))
            neighboors=np.where(find_neighboor(puzzle_grid,np.array(position))==1)[0]
            if np.shape(neighboors)[0]!=0:
                target=np.random.choice(neighboors,None,False)
                if True:
                    m=target//2
                    zero_array=np.zeros(3)
                    if target%2==0:
                        zero_array[m]=1
                    else:
                        zero_array[m]=-1
                    target_position=tuple(np.add(position,zero_array).astype(int))
                    puzzle_grid[target_position]=k
                    dic_pieces["Piece_{}".format(k)]=[position,target_position]
            else :
                sucess_2_cube=False
        if sucess_2_cube==True:
            missing_pieces=(size_Puzzle**3)-2*number_Of_Pieces
            while missing_pieces!=0:
                a=np.arange(1,number_Of_Pieces+1)
                np.random.shuffle(a)
                for k in a:
                    if np.random.random()>=0.2:
                        eaten=False
                        positions=dic_pieces["Piece_{}".format(k)]
                        np.random.shuffle(positions)
                        for position in positions:
                            if eaten==False:
                                neighboors=np.where(find_neighboor(puzzle_grid,np.array(position))==1)[0]
                                if np.shape(neighboors)[0]!=0:
                                    target=np.random.choice(neighboors,None,False)
                                    m=target//2
                                    zero_array=np.zeros(3)
                                    if target%2==0:
                                        zero_array[m]=1
                                    else:
                                        zero_array[m]=-1
                                    target_position=tuple(np.add(position,zero_array).astype(int))
                                    puzzle_grid[target_position]=k
                                    missing_pieces-=1
                                    #print(missing_pieces)
                                    eaten=True
                                    dic_pieces["Piece_{}".format(k)].append(target_position)
                                else:
                                    continue
                    else:
                        continue
    return puzzle_grid,dic_pieces

def generate_puzzle_with_stat(size_Puzzle : int,number_Of_Pieces:int,std_size_obj:float,std_size_inter:float):
    """Generate a puzzle and verify its statistical properties"""
    good_gen=False
    while good_gen!=True:
        grid,dic=generate_puzzle(size_Puzzle,number_Of_Pieces)
        mean_size,std_size=check_stat(dic)
        if (std_size_obj-std_size_inter<=std_size) and (std_size<=std_size_obj+std_size_inter):
            good_gen=True
    return grid,dic

def generate_json(dir : str,size_Puzzle : int,number_Of_Pieces:int,std_size_obj:float,std_size_inter:float):
    """Write 200 statistically verified puzzle grids in a json file (dir)
    """
    L=[]
    for k in range(0,200):
        grid,dic=generate_puzzle_with_stat(size_Puzzle,number_Of_Pieces,std_size_obj,std_size_obj)
        L.append(grid.tolist())
    with open(dir, "w") as write_file:
        json.dump(L, write_file)

In [4]:
generate_json("puzzle_grid_data.json",4,11,1.6,0.1)